In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1144075,2021-04-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1144076,2021-04-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1144077,2021-04-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1144078,2021-04-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
30739,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30741,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30743,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30745,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30747,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
705797,2021-04-02,Arkansas,Arkansas,5001,2077,36.00,5000,Arkansas,AR,Arkansas,State,3017804
705799,2021-04-03,Arkansas,Arkansas,5001,2079,36.00,5000,Arkansas,AR,Arkansas,State,3017804
705801,2021-04-04,Arkansas,Arkansas,5001,2079,36.00,5000,Arkansas,AR,Arkansas,State,3017804
705803,2021-04-05,Arkansas,Arkansas,5001,2081,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1144075,2021-04-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1144076,2021-04-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1144077,2021-04-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1144078,2021-04-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-06') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
441,2021-04-06,Snohomish,Washington,33271,563.00,53061,WA,County,822083,4047.16,68.48
880,2021-04-06,Cook,Illinois,505380,10264.00,17031,IL,County,5150233,9812.76,199.29
1318,2021-04-06,Orange,California,267143,4784.00,6059,CA,County,3175692,8412.12,150.64
1755,2021-04-06,Maricopa,Arizona,526415,9670.00,4013,AZ,County,4485414,11736.15,215.59
2192,2021-04-06,Los Angeles,California,1223435,23302.00,6037,CA,County,10039107,12186.69,232.11
...,...,...,...,...,...,...,...,...,...,...,...
1143499,2021-04-06,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1143675,2021-04-06,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1143820,2021-04-06,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1143961,2021-04-06,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
441,2021-04-06,Snohomish,Washington,33271,563.00,53061,WA,County,822083,4047.16,68.48,68.48,4047.16
880,2021-04-06,Cook,Illinois,505380,10264.00,17031,IL,County,5150233,9812.76,199.29,199.29,9812.76
1318,2021-04-06,Orange,California,267143,4784.00,6059,CA,County,3175692,8412.12,150.64,150.64,8412.12
1755,2021-04-06,Maricopa,Arizona,526415,9670.00,4013,AZ,County,4485414,11736.15,215.59,215.59,11736.15
2192,2021-04-06,Los Angeles,California,1223435,23302.00,6037,CA,County,10039107,12186.69,232.11,232.11,12186.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143499,2021-04-06,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1143675,2021-04-06,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1143820,2021-04-06,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1143961,2021-04-06,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
737966,2021-04-06,Hale,Alabama,2178,74.00,1065,AL,County,14651,14865.88,505.08,505.08,14865.88,1
803161,2021-04-06,Clarke,Alabama,3456,60.00,1025,AL,County,23622,14630.43,254.00,254.00,14630.43,2
577581,2021-04-06,Lowndes,Alabama,1362,53.00,1085,AL,County,9726,14003.70,544.93,544.93,14003.70,3
429876,2021-04-06,Franklin,Alabama,4190,82.00,1059,AL,County,31362,13360.12,261.46,261.46,13360.12,4
512193,2021-04-06,Etowah,Alabama,13653,345.00,1055,AL,County,102268,13350.22,337.35,337.35,13350.22,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085748,2021-04-06,Platte,Wyoming,611,11.00,56031,WY,County,8393,7279.88,131.06,131.06,7279.88,19
704308,2021-04-06,Converse,Wyoming,984,17.00,56009,WY,County,13822,7119.09,122.99,122.99,7119.09,20
875313,2021-04-06,Lincoln,Wyoming,1373,12.00,56023,WY,County,19830,6923.85,60.51,60.51,6923.85,21
973949,2021-04-06,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
577581,2021-04-06,Lowndes,Alabama,1362,53.00,1085,AL,County,9726,14003.70,544.93,544.93,14003.70,3,1
737966,2021-04-06,Hale,Alabama,2178,74.00,1065,AL,County,14651,14865.88,505.08,505.08,14865.88,1,2
245854,2021-04-06,Walker,Alabama,7079,274.00,1127,AL,County,63521,11144.35,431.35,431.35,11144.35,28,3
622818,2021-04-06,Greene,Alabama,906,34.00,1063,AL,County,8111,11170.02,419.18,419.18,11170.02,27,4
576827,2021-04-06,Crenshaw,Alabama,1497,57.00,1041,AL,County,13772,10869.88,413.88,413.88,10869.88,31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621313,2021-04-06,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
875313,2021-04-06,Lincoln,Wyoming,1373,12.00,56023,WY,County,19830,6923.85,60.51,60.51,6923.85,21,20
853542,2021-04-06,Uinta,Wyoming,2135,12.00,56041,WY,County,20226,10555.72,59.33,59.33,10555.72,4,21
244702,2021-04-06,Teton,Wyoming,3640,9.00,56039,WY,County,23464,15513.13,38.36,38.36,15513.13,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141387,2021-04-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1141388,2021-04-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1141389,2021-04-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1141390,2021-04-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
734906,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,505.08,14865.88,2,1,1.00
734907,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,505.08,14865.88,2,1,0.00
734908,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,505.08,14865.88,2,1,0.00
734909,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,505.08,14865.88,2,1,0.00
734910,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,505.08,14865.88,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889917,2021-04-02,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
889918,2021-04-03,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
889919,2021-04-04,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
889920,2021-04-05,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
575267,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,1.00,0.00
575268,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,0.00,0.00
575269,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,0.00,0.00
575270,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,0.00,0.00
575271,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573377,2021-04-02,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10303.50,23,5,5.00,0.00
573378,2021-04-03,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10303.50,23,5,0.00,0.00
573379,2021-04-04,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10303.50,23,5,0.00,0.00
573380,2021-04-05,Albany,Wyoming,3990,11.00,56001,WY,County,38880,10262.35,28.29,28.29,10303.50,23,5,6.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-06') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
293438,2021-04-06,Imperial,California,27604,711.00,6025,CA,County,181215,15232.73,392.35,392.35,15232.73,1,2,16.00,0.00
2192,2021-04-06,Los Angeles,California,1223435,23302.00,6037,CA,County,10039107,12186.69,232.11,232.11,12186.69,2,5,331.00,1.00
146208,2021-04-06,San Bernardino,California,291728,4425.00,6071,CA,County,2180085,13381.50,202.97,202.97,13381.50,3,4,1.00,2.00
513657,2021-04-06,Inyo,California,1377,36.00,6027,CA,County,18039,7633.46,199.57,199.57,7633.46,4,25,0.00,0.00
65425,2021-04-06,Stanislaus,California,59536,1007.00,6099,CA,County,550660,10811.75,182.87,182.87,10811.75,5,9,76.00,3.00
30342,2021-04-06,Riverside,California,295631,4417.00,6065,CA,County,2470546,11966.22,178.79,178.79,11966.22,6,6,205.00,28.00
81102,2021-04-06,Tulare,California,49175,818.00,6107,CA,County,466195,10548.16,175.46,175.46,10548.16,7,10,46.00,0.00
52065,2021-04-06,San Joaquin,California,70465,1325.00,6077,CA,County,762148,9245.58,173.85,173.85,9245.58,8,19,220.00,12.00
29550,2021-04-06,Fresno,California,99755,1622.00,6019,CA,County,999101,9984.48,162.35,162.35,9984.48,9,14,53.00,0.00
465541,2021-04-06,Merced,California,30939,442.00,6047,CA,County,277680,11141.96,159.18,159.18,11141.96,10,8,33.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1085299,2021-04-06,Lassen,California,5660,1.00,6035,CA,County,30573,18513.07,3.27,3.27,18513.07,56,1,4.00,0.00
293438,2021-04-06,Imperial,California,27604,711.00,6025,CA,County,181215,15232.73,392.35,392.35,15232.73,1,2,16.00,0.00
622760,2021-04-06,Kings,California,22674,242.00,6031,CA,County,152940,14825.42,158.23,158.23,14825.42,11,3,5.00,1.00
146208,2021-04-06,San Bernardino,California,291728,4425.00,6071,CA,County,2180085,13381.50,202.97,202.97,13381.50,3,4,1.00,2.00
2192,2021-04-06,Los Angeles,California,1223435,23302.00,6037,CA,County,10039107,12186.69,232.11,232.11,12186.69,2,5,331.00,1.00
30342,2021-04-06,Riverside,California,295631,4417.00,6065,CA,County,2470546,11966.22,178.79,178.79,11966.22,6,6,205.00,28.00
183789,2021-04-06,Kern,California,106543,1292.00,6029,CA,County,900202,11835.45,143.52,143.52,11835.45,14,7,63.00,1.00
465541,2021-04-06,Merced,California,30939,442.00,6047,CA,County,277680,11141.96,159.18,159.18,11141.96,10,8,33.00,0.00
65425,2021-04-06,Stanislaus,California,59536,1007.00,6099,CA,County,550660,10811.75,182.87,182.87,10811.75,5,9,76.00,3.00
81102,2021-04-06,Tulare,California,49175,818.00,6107,CA,County,466195,10548.16,175.46,175.46,10548.16,7,10,46.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
293438,2021-04-06,Imperial,California,27604,711.00,6025,CA,County,181215,15232.73,392.35,392.35,15232.73,1,2,16.00,0.00
2192,2021-04-06,Los Angeles,California,1223435,23302.00,6037,CA,County,10039107,12186.69,232.11,232.11,12186.69,2,5,331.00,1.00
146208,2021-04-06,San Bernardino,California,291728,4425.00,6071,CA,County,2180085,13381.50,202.97,202.97,13381.50,3,4,1.00,2.00
513657,2021-04-06,Inyo,California,1377,36.00,6027,CA,County,18039,7633.46,199.57,199.57,7633.46,4,25,0.00,0.00
65425,2021-04-06,Stanislaus,California,59536,1007.00,6099,CA,County,550660,10811.75,182.87,182.87,10811.75,5,9,76.00,3.00
30342,2021-04-06,Riverside,California,295631,4417.00,6065,CA,County,2470546,11966.22,178.79,178.79,11966.22,6,6,205.00,28.00
81102,2021-04-06,Tulare,California,49175,818.00,6107,CA,County,466195,10548.16,175.46,175.46,10548.16,7,10,46.00,0.00
52065,2021-04-06,San Joaquin,California,70465,1325.00,6077,CA,County,762148,9245.58,173.85,173.85,9245.58,8,19,220.00,12.00
29550,2021-04-06,Fresno,California,99755,1622.00,6019,CA,County,999101,9984.48,162.35,162.35,9984.48,9,14,53.00,0.00
465541,2021-04-06,Merced,California,30939,442.00,6047,CA,County,277680,11141.96,159.18,159.18,11141.96,10,8,33.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15232.73,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15232.73,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15232.73,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15232.73,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15232.73,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3842,04/02/21,Lassen,5652,1.00,18486.90,3.27,3.27,18513.07,56,1,0.00,0.00
3843,04/03/21,Lassen,5652,1.00,18486.90,3.27,3.27,18513.07,56,1,0.00,0.00
3844,04/04/21,Lassen,5652,1.00,18486.90,3.27,3.27,18513.07,56,1,0.00,0.00
3845,04/05/21,Lassen,5656,1.00,18499.98,3.27,3.27,18513.07,56,1,4.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15232.73,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15232.73,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15232.73,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15232.73,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15232.73,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3928,04/02/21,Merced,30687,439.00,11051.21,158.10,159.18,11141.96,10,8,38.00,1.00
3929,04/03/21,Merced,30687,439.00,11051.21,158.10,159.18,11141.96,10,8,0.00,0.00
3930,04/04/21,Merced,30687,439.00,11051.21,158.10,159.18,11141.96,10,8,0.00,0.00
3931,04/05/21,Merced,30906,442.00,11130.08,159.18,159.18,11141.96,10,8,219.00,3.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)